preliminary stuff

In [3]:
import pandas as pd

df = pd.read_pickle('sokureplays.pkl')
characters = ['aya', 'youmu', 'iku', 'reimu', 'reisen', 'remilia', 'marisa',
       'suika', 'tenshi', 'yuyuko', 'komachi', 'suwako', 'sakuya',
       'utsuho', 'meiling', 'yukari', 'patchouli', 'alice', 'cirno',
       'sanae']

In [26]:
df.count()

id                  2163674
server_player_id    2163674
client_player_id    2163674
time                2163674
server_rounds       2163674
client_rounds       2163674
server_character    2163674
client_character    2163674
server_cards        2163674
client_cards        2163674
unranked            2163674
server_nick         2163674
client_nick         2163674
server_country      2163674
client_country      2163674
dtype: int64

In [4]:
df.columns

Index(['id', 'server_player_id', 'client_player_id', 'time', 'server_rounds',
       'client_rounds', 'server_character', 'client_character', 'server_cards',
       'client_cards', 'unranked', 'server_nick', 'client_nick',
       'server_country', 'client_country'],
      dtype='object')

In [5]:
df.query("client_rounds == 2").count()

id                  970419
server_player_id    970419
client_player_id    970419
time                970419
server_rounds       970419
client_rounds       970419
server_character    970419
client_character    970419
server_cards        970419
client_cards        970419
unranked            970419
server_nick         970419
client_nick         970419
server_country      970419
client_country      970419
dtype: int64

next 2 blocks count non mirror total games and wins per char

In [107]:
character_total_games = []
for character in characters:
    count = df.query(f"(server_character == '{character}' | client_character == '{character}') & server_character != client_character").count()
    character_total_games.append(int(count['id']))

In [109]:
character_total_wins = []
for character in characters:
    count = df.query(f"(server_character != client_character) & ((server_rounds == 2 & server_character == '{character}') | (client_rounds == 2 & client_character == '{character}'))").count()
    character_total_wins.append(int(count['id']))

prints win rate per char

In [ ]:
for character in characters:
    print(f"{character}: {character_total_wins[characters.index(character)]}/{character_total_games[characters.index(character)]}. Win Rate: {character_total_wins[characters.index(character)] / character_total_games[characters.index(character)] * 100}%")

creates dataframes for mirror matches and non mirror matches

In [ ]:
df_no_mirror_matches = df.query("server_character != client_character")
df_mirror_matches = df.query("server_character == client_character")
df_mirror_matches['server_character'].value_counts()

Counts the most common server characters and client characters:

In [ ]:
df['server_character'].value_counts()
#df['client_character'].value_counts()

build dataframes for each character

In [4]:
character_data_frames = []
for character in characters:
    new_df = df.query(f"(server_character == '{character}' | client_character == '{character}') & server_character != client_character")
    character_data_frames.append(new_df)

counts how many unique players a character has (rename variables later)

In [5]:
for character in characters:
    reimu_df = character_data_frames[characters.index(character)]
    all_players = pd.concat([reimu_df['server_nick'], reimu_df['client_nick']])
    unique_players = all_players.unique()
    player_list = list(unique_players)
    print(f"Number of unique players for {character}: {len(player_list)}")

Number of unique players for aya: 4759
Number of unique players for youmu: 5647
Number of unique players for iku: 4234
Number of unique players for reimu: 5048
Number of unique players for reisen: 5262
Number of unique players for remilia: 5423
Number of unique players for marisa: 5310
Number of unique players for suika: 5313
Number of unique players for tenshi: 4695
Number of unique players for yuyuko: 4799
Number of unique players for komachi: 4923
Number of unique players for suwako: 4731
Number of unique players for sakuya: 5981
Number of unique players for utsuho: 5284
Number of unique players for meiling: 5182
Number of unique players for yukari: 4794
Number of unique players for patchouli: 4974
Number of unique players for alice: 4689
Number of unique players for cirno: 4882
Number of unique players for sanae: 4147


checking who contributes to character's wr the most
distinguish fix server vs client 

In [ ]:
char_player_df_list = []
for character in characters:
    print("doing stuff for " + character)
    chars_own_df = character_data_frames[characters.index(character)]
    players_of_char = pd.concat([chars_own_df['server_nick'], chars_own_df['client_nick']])
    unique_players = players_of_char.unique()
    player_list = list(unique_players)
    print(f"Number of unique players for {character}: {len(player_list)}")
    
    char_df = pd.DataFrame(columns=['Player', 'Total Wins', 'Total Games', 'WR Contribution %'])
    i = 0
    for player in player_list:
        i = i+1
        print(f"{i}/{len(player_list)}")
        players_df = chars_own_df.query(f'''(server_nick == "{player}" & server_character == "{character}") | (client_nick == "{player}" & client_character == "{character}")''')
        counted = players_df.count()
        player_total_games = int(counted['id'])
        player_wins = players_df.query(f'''(server_rounds == 2 & server_nick == "{player}") | (client_rounds == 2 & client_nick == "{player}")''').count()
        player_total_wins = int(player_wins['id'])
        char_df = char_df._append({'Player': player, 'Total Wins': player_total_wins, 'Total Games': player_total_games}, ignore_index=True)
        
        
    char_df["WR Contribution %"] = 0.0
    char_global_wr = character_total_wins[characters.index(character)] / character_total_games[characters.index(character)]
        
    for index, row in char_df.iterrows():
        what_if_win = character_total_wins[characters.index(character)] - row['Total Wins']
        what_if_games = character_total_games[characters.index(character)] - row['Total Games']
        what_if_winrate = what_if_win / what_if_games
        char_df.at[index, "WR Contribution %"] = (char_global_wr - what_if_winrate) * 100
        
    char_df.sort_values(by='WR Contribution %', ascending=False)    
    char_player_df_list.append(char_df)
    

In [526]:
#pickle the player dataframes
for i in range(len(char_player_df_list)):
    char_player_df_list[i].to_pickle(f'{characters[i]}.pkl')

print wr contribution data

In [2]:
i = 11
#2.37 at 2
print(f"{characters[i]}")
temp = char_player_df_list[i].sort_values(by='WR Contribution %', ascending=False).tail(6)
temp.drop(index=temp.index[0], axis=0, inplace=True)
temp.style.hide()

NameError: name 'characters' is not defined

In [522]:
df_no_mirror_matches.query("client_character == 'youmu' & server_nick == 'chickychan'").count()

id                  1028
server_player_id    1028
client_player_id    1028
time                1028
server_rounds       1028
client_rounds       1028
server_character    1028
client_character    1028
server_cards        1028
client_cards        1028
unranked            1028
server_nick         1028
client_nick         1028
server_country      1028
client_country      1028
dtype: int64